### Necessary imports

# Programming Assignment: Social Network Database Fix

## Welcome!

You've reached a practical part of our course, **Generative AI for Software Development - AI-Powered Software and System Design**. This task is all about diving into a database designed for managing a social network. It's a hands-on challenge to test your skills and see how well you can work with AI tools, like a Language Learning Model (LLM).

## Your Mission

Your job is to fix a database code that's not working right. It's supposed to handle a social network with two main parts: a `Person` table and a `Club` table. There's a mistake in the code, and it's up to you to find and correct it.

### Tasks:

1. **Fix the Database:** Hunt down the mistake messing up the database and correct it.
2. **Write Three Functions:** Use the LLM to help you craft three functions. They should:
   - List all members of a specific club.
   - List all friends of a specific person.
   - List all people who consider a specific person their friend.

### Working with the LLM:

- **Ask the LLM for Help:** Use the LLM to guide you through fixing the database and creating your functions.
- **Use Its Advice Wisely:** Remember, the LLM's advice might not always be spot-on. It's up to you to decide what's useful.

## What to Submit:

Please submit this Jupyter notebook containing:
- The corrected database code.
- The three functions you've written.

## How We'll Grade It:

- **Database Accuracy:** We'll check if you've successfully identified and fixed the issue in the database.
- **Functionality of Your Functions:** We'll see if each of your functions is doing exactly what it's supposed to do.

## Tips for Success:

- **Be Clear with Your Questions:** The clearer your questions to the LLM, the better help you'll get.
- **Test the Advice:** Always test out the LLM’s suggestions to ensure they work as expected.
- **Hints**: If you struggle with this assignment, you can check some hints we left to you in the bottom of the assignment!

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
import numpy as np
import os

In [2]:
import unittests

## The main code

Below is the primary code for the assignment. It contains **two significant flaws** that could affect the results of certain functions you'll develop for this task. Identifying the bugs directly from the code might be challenging for an LLM, **so you might need to implement some of the functions to better understand the issues**. Your analytical skills will be crucial. Note that although you are required to create three functions, your submission will be evaluated on four aspects, one of which is the accuracy of your database. Ensure that you address the issues in the code provided!

**If you need to start over with a clean version of this assignment, there's a folder named `backup_data` where you can find a fresh copy.**

In [3]:

def load_dataset(path = "./"):
    """Loads the dataset
    
    RUN THE FUNCTION WITHOUT ANY PARAMETER. 
    THE PARAMETER IS FOR GRADING PURPOSES ONLY.
    
    """
    
    # Ensure reproducibility, but randomness will be removed from selections
    np.random.seed(42)
    
    # Remove existing database if it exists
    if 'social_network.db' in os.listdir(path):
        os.remove('social_network.db')
    
    # Setup engine and define tables
    engine = create_engine(f'sqlite:///{os.path.join(path, "social_network.db")}', echo=False)
    Base = declarative_base()
    
    # Define the friendship association table
    friendships = Table('friendships', Base.metadata,
        Column('person_id', Integer, ForeignKey('people.id'), primary_key=True),
        Column('friend_id', Integer, ForeignKey('people.id'), primary_key=True)
    )
    
    class Person(Base):
        __tablename__ = 'people'
        id = Column(Integer, primary_key=True)
        name = Column(String)
        age = Column(Integer)
        gender = Column(String)
        location = Column(String)
    
        friends = relationship("Person",
                               secondary=friendships,
                               primaryjoin=(friendships.c.person_id == id),
                               secondaryjoin=(friendships.c.friend_id == id),
                               backref="friend_of")
        clubs = relationship("Club", secondary="club_members")
    
    class Club(Base):
        __tablename__ = 'clubs'
        id = Column(Integer, primary_key=True)
        description = Column(String)
    
        members = relationship("Person", secondary="club_members")
    
    club_members = Table('club_members', Base.metadata,
        Column('person_id', Integer, ForeignKey('people.id')),
        Column('club_id', Integer, ForeignKey('clubs.id'))
    )
    
    # Create tables
    Base.metadata.create_all(engine)
    
    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()
    
    # Sample data
    names = np.array([("Alice", "New York", "Non-binary", 30), 
             ("Bob", "Los Angeles", "Male", 18), 
             ("Charlie", "Chicago", "Male", 60), 
             ("David", "Houston", "Male", 59),
             ("Eve", "Phoenix", "Non-binary", 18), 
             ("Frank", "Los Angeles", "Non-binary", 72), 
             ("Grace", "Chicago", "Female", 35), 
             ("Henry", "Houston", "Male", 21), 
             ("Ivy", "New York", "Female", 46), 
             ("Elena", "Phoenix", "Female", 66)])
    club_descriptions = [
        "Book Club", "Hiking Club", "Chess Club", "Photography Club", "Cooking Club",
        "Music Club", "Gaming Club", "Fitness Club", "Art Club", "Travel Club"
    ]
    
    # Populate Person table with the 10 unique persons only
    people = []
    for name, location, gender, age in names:
        person = Person(name=name, age=age, gender=gender, location=location)
        people.append(person)
        session.add(person)
    
    # Populate Friendships (one-to-one relationships based on the order to avoid randomness)
    for i, person in enumerate(people):
        if i < len(people) - 1:
            person.friends.append(people[i + 1])
        else:
            person.friends.append(people[0])  # Loop back to the first person
    
    # Populate Clubs
    clubs = []
    for description in club_descriptions:
        club = Club(description=description)
        clubs.append(club)
        session.add(club)
    
    # Populate Club Members (assign each person to a unique club, avoiding randomness)
    for i, person in enumerate(people):
        clubs[i % len(clubs)].members.append(person)
    
    # Commit and close session
    session.commit()
    session.close()
    
    print("Database created and populated successfully with deterministic data!")
    return session, Club, Person, friendships


In [4]:
# Loads the dataset
session, Club, Person, friendships = load_dataset()

/tmp/ipykernel_97/3290300261.py:20: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Database created and populated successfully!


/tmp/ipykernel_97/3290300261.py:81: SAWarning: relationship 'Club.members' will copy column clubs.id to column club_members.club_id, which conflicts with relationship(s): 'Person.clubs' (copies clubs.id to club_members.club_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="clubs"' to the 'Club.members' relationship. (Background on this warning at: https://sqlalche.me/e/20/qzyx) (This warning originated from the `configure_mappers()` process, which was invoked automatically in response to a user-initiated operation.)
  person = Person(
/tmp/ipykernel_97/3290300261.py:81: SAWarning: relationship 'Club.members' will copy column peo

### Exercise 1

For this exercise, you are tasked with creating a function named `get_club_members_by_description`. This function should accept a description of a club and a session, and return a list of all its members. **Ensure that this function returns a list containing the defined Person objects.** It must input only a **club description**.

In [5]:
def get_club_members(session,club_description):
    """
    Returns a list of Person objects who are members of a club given the club's description.
    
    Parameters:
    - club_description (str): The description of the club for which members are to be retrieved.
    
    Returns:
    - List[Person]: A list of Person objects who are members of the specified club.
    """
    # Query the Club table to get the club with the given description
    club = session.query(Club).filter(Club.description == club_description).first()
    
    # If the club does not exist, return an empty list
    if club is None:
        return []
    
    # Return the list of members associated with the found club
    return club.members

In [6]:
# Example usage of the get_club_members function

# Assume the session and all models have been correctly set up and populated as per your initial code

# Fetching members of the "Hiking Club"
hiking_club_members = get_club_members(session,"Hiking Club")

# Printing out the names of all members of the Hiking Club
print("Members of the Hiking Club:")
for person in hiking_club_members:
    print(f"- {person.name}, Age: {person.age}, Location: {person.location}")

Members of the Hiking Club:
- David, Age: 72, Location: Phoenix
- Grace, Age: 59, Location: Chicago
- Henry, Age: 60, Location: Phoenix
- Alice, Age: 46, Location: Los Angeles
- Ivy, Age: 35, Location: Houston
- Ivy, Age: 59, Location: Houston


In [7]:
unittests.test_get_club_members(load_dataset, get_club_members)

Failed test case: Incorrect club members for club description Book Club.
Expected: ['Alice', 'Charlie', 'Elena', 'Eve', 'Frank', 'Grace']
Got: ['Bob', 'Charlie', 'Charlie', 'Elena', 'Eve', 'Eve', 'Grace', 'Henry', 'Henry']

Failed test case: Incorrect club members for club description Hiking Club.
Expected: ['Alice', 'David', 'Elena', 'Eve', 'Frank', 'Ivy']
Got: ['Alice', 'David', 'Grace', 'Henry', 'Ivy', 'Ivy']

Failed test case: Incorrect club members for club description Chess Club.
Expected: ['Alice', 'David', 'Elena', 'Eve', 'Frank', 'Grace']
Got: ['Alice', 'Bob', 'Charlie', 'Eve', 'Eve', 'Frank', 'Henry']

Failed test case: Incorrect club members for club description Photography Club.
Expected: ['Alice', 'Charlie', 'David', 'Elena']
Got: ['Alice', 'Alice', 'Bob', 'Bob', 'Charlie', 'Eve', 'Grace', 'Grace', 'Henry']

Failed test case: Incorrect club members for club description Cooking Club.
Expected: ['Alice', 'Bob', 'Charlie', 'David', 'Grace', 'Henry']
Got: ['Alice', 'Charlie', 

### Exercise 2

In this exercise, you are required to create a function named `get_friends_of_person`. This function should accept the name of a person and a session, return a list of all their friends. **Ensure that this function returns a list containing the defined Person objects.** The input must be only the **name of a person**.

In [8]:
def get_friends_of_person(session, person_name):
    """
    Returns a list of Person objects who are friends with the specified person.
    
    Parameters:
    - person_name (str): The name of the person for whom to retrieve friends.
    
    Returns:
    - List[Person]: A list of Person objects who are friends with the specified person.
    """
    # Query the Person table to find the person with the given name
    person = session.query(Person).filter(Person.name == person_name).first()
    
    # If the person does not exist, return an empty list
    if person is None:
        return []
    
    # Return the list of friends associated with the found person
    return person.friends

In [9]:
# Example usage of the get_friends_of_person function

# Fetching friends of given name
name = "Bob"

alice_friends = get_friends_of_person(session,name)

# Printing out the names of all friends of Alice
print(f"Friends of {name}:")
for friend in alice_friends:
    print(f"- {friend.name}, Age: {friend.age}, Location: {friend.location}")

Friends of Bob:
- Grace, Age: 35, Location: Chicago


In [10]:
unittests.test_get_friends_of_person(load_dataset, get_friends_of_person)

Failed test case: Incorrect friends for person Alice.
Expected: ['Bob', 'Charlie', 'David', 'Elena', 'Eve', 'Henry', 'Ivy']
Got: ['Alice', 'David']

Failed test case: Incorrect friends for person Bob.
Expected: ['Alice', 'David', 'Elena', 'Eve', 'Frank', 'Henry', 'Ivy']
Got: ['Grace']

Failed test case: Incorrect friends for person Charlie.
Expected: ['Alice', 'Bob', 'David', 'Elena', 'Frank', 'Grace', 'Henry', 'Ivy']
Got: ['Eve', 'Grace']

Failed test case: Incorrect friends for person David.
Expected: ['Alice', 'Bob', 'Charlie', 'Elena', 'Eve', 'Frank', 'Grace', 'Henry', 'Ivy']
Got: ['Bob', 'Bob']

Failed test case: Incorrect friends for person Eve.
Expected: ['Alice', 'Charlie', 'David', 'Elena', 'Frank', 'Grace', 'Henry']
Got: ['Elena', 'Eve']

Failed test case: Incorrect friends for person Frank.
Expected: ['Alice', 'Bob', 'Charlie', 'David', 'Elena', 'Eve', 'Grace', 'Henry']
Got: ['Eve']

Failed test case: Incorrect friends for person Grace.
Expected: ['Alice', 'Bob', 'David', 'E

### Exercise 3

In this exercise, you're tasked with crafting a function called `get_persons_who_consider_them_friend`. This function should take two parameters: the name of an individual and a session. It will return a list of people who count this individual as a friend. It's important to remember that in our database, friendship isn't necessarily mutual. For example, Alice might consider Bob a friend, but Bob might not feel the same way about Alice. **Your function must return a list of Person objects for everyone who considers the input name their friend.** The input to this function should strictly be the **name of the person** you're inquiring about.

In [11]:
def get_persons_who_consider_them_friend(session, person_name):
    """
    Returns a list of Person objects who consider the specified person as their friend,
    in a scenario where friendships are unidirectional.
    
    Parameters:
    - person_name (str): The name of the person to find who is considered as a friend by others.
    
    Returns:
    - List[Person]: A list of Person objects who consider the specified person as their friend.
    """
    # First, find the person by the given name to get their ID
    person = session.query(Person).filter(Person.name == person_name).first()
    
    # If the person does not exist, return an empty list
    if person is None:
        return []
    
    # Query the friendships table for rows where the friend_id matches the person's ID
    # Then, join with the Person table to get the details of the people who consider them a friend
    friends_of = session.query(Person).join(friendships, Person.id == friendships.c.person_id).filter(friendships.c.friend_id == person.id).all()
    
    return friends_of

In [12]:
# Example usage of the get_persons_who_consider_them_friend function

# Fetching people who consider given name as their friend
name = 'Bob'

name_friend_of = get_persons_who_consider_them_friend(session, name)

# Printing out the names of all people who consider Alice as their friend
print(f"People who consider {name} as their friend:")
for person in name_friend_of:
    print(f"- {person.name}, Age: {person.age}, Location: {person.location}")

People who consider Bob as their friend:
- David, Age: 18, Location: Chicago
- Bob, Age: 18, Location: Chicago
- Grace, Age: 59, Location: New York
- Frank, Age: 35, Location: Phoenix


In [13]:
unittests.test_get_persons_who_consider_them_friend(load_dataset, get_persons_who_consider_them_friend)

Failed test case: Incorrect persons who consider Alice a friend.
Expected: ['Bob', 'Charlie', 'David', 'Elena', 'Eve', 'Frank', 'Grace', 'Henry', 'Ivy']
Got: ['Ivy']

Failed test case: Incorrect persons who consider Bob a friend.
Expected: ['Alice', 'Charlie', 'David', 'Frank', 'Grace', 'Henry', 'Ivy']
Got: ['Bob', 'David', 'Frank', 'Grace']

Failed test case: Incorrect persons who consider Charlie a friend.
Expected: ['Alice', 'David', 'Elena', 'Eve', 'Frank', 'Henry', 'Ivy']
Got: ['Bob', 'Elena']

Failed test case: Incorrect persons who consider David a friend.
Expected: ['Alice', 'Bob', 'Charlie', 'Elena', 'Eve', 'Frank', 'Grace', 'Henry', 'Ivy']
Got: ['Frank']

Failed test case: Incorrect persons who consider Eve a friend.
Expected: ['Alice', 'Bob', 'David', 'Elena', 'Frank', 'Grace', 'Henry', 'Ivy']
Got: ['Ivy']

Failed test case: Incorrect persons who consider Frank a friend.
Expected: ['Bob', 'Charlie', 'David', 'Elena', 'Eve', 'Grace', 'Ivy']
Got: ['Eve', 'Frank']

Failed test 

## Now test your Dataset!

In [ ]:
unittests.test_load_dataset(load_dataset)

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint 1</b></font>
</summary>
<p>
Check how the persons are being inserted into the dataset! Does the random is necessary?
</p>
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint 2</b></font>
</summary>
<p>
You may have to properly handle how friendships are created. Ask an LLM how you can fix that part.
</p>
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint 3</b></font>
</summary>
<p>
Remember that friendships are not bidirectional! Specify it when asking the LLM to make your last function!
</p>
</details>

Congratulations! You have finished the assignment! Keep up!